In [2]:
import pandas as pd
import numpy as np

In [97]:
df_data1 = ['A'] * 7 + ['B'] * 3 + ['C'] * 5 + ['D'] * 5
df_data2 = [0] * 20

In [98]:
df_data = pd.DataFrame({'col1': df_data1, 'col2': df_data2})

In [30]:
b = np.cumsum(df_data.value_counts(normalize=True).values)

In [31]:
a = df_data.value_counts(normalize=True).index.get_level_values(0).to_list()

In [40]:
b[-1] = 1

In [41]:
b

array([0.35, 0.6 , 0.85, 1.  ])

In [45]:
c = np.roll(b, 1)
c[0] = 0.0

In [57]:
x= list(zip(c, b))

In [58]:
x

[(0.0, 0.35), (0.35, 0.6), (0.6, 0.85), (0.85, 1.0)]

In [63]:
d = dict(zip(a, x))

In [64]:
d['A']

(0.0, 0.35)

In [80]:
df_t = df_data['A'].map(lambda x: np.random.uniform(d[x][0], d[x][1], size=1)[0])

In [82]:
type(df_t)

pandas.core.series.Series

In [87]:
df_t

0     0.305603
1     0.022187
2     0.306261
3     0.247311
4     0.210021
5     0.273874
6     0.040238
7     0.895683
8     0.897454
9     0.898481
10    0.558260
11    0.420451
12    0.508663
13    0.509995
14    0.569979
15    0.750503
16    0.808595
17    0.838406
18    0.662726
19    0.633094
Name: A, dtype: float64

In [94]:
c1 = np.append(c, 1)

In [96]:
pd.cut(df_t, right=False, include_lowest=True, bins=c1, labels=a, ordered=False)

0     A
1     A
2     A
3     A
4     A
5     A
6     A
7     B
8     B
9     B
10    C
11    C
12    C
13    C
14    C
15    D
16    D
17    D
18    D
19    D
Name: A, dtype: category
Categories (4, object): ['A', 'C', 'D', 'B']

In [99]:
from sklearn.preprocessing import LabelEncoder

In [100]:
le = LabelEncoder()

In [103]:
le.fit_transform(df_data1)

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3])

In [112]:
df_t.dtypes == np.float64

True

In [113]:
df_data['col1'].values

array(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C',
       'C', 'C', 'D', 'D', 'D', 'D', 'D'], dtype=object)

In [11]:
df_data = pd.DataFrame({'A': [0.0] * 10000 + [10000.0], 'B': pd.to_datetime(['2222-12-12']).append(pd.to_datetime(['2022-12-13']*10000))})

In [12]:
df_data

,A,B
0,0.0,2222-12-12
1,0.0,2022-12-13
2,0.0,2022-12-13
3,0.0,2022-12-13
4,0.0,2022-12-13
...,...,...
9996,0.0,2022-12-13
9997,0.0,2022-12-13
9998,0.0,2022-12-13
9999,0.0,2022-12-13
